In [1]:
import numpy as np
from mpm_functions import MPMGrid1D, ElasticMaterial1D, MPMBody1D
import plotly.graph_objects as go

In [2]:
# Setup
dt           = 1e-4         # s
TrussLength  = 0.2          # m
CrossSection = 0.02 * 0.02  # m^2
GridLength   = 1            # m
NoCells      = 21
NoMP         = 25
# Grid
Grid = MPMGrid1D(0,GridLength,NoCells)
# Material
Emod = 1e6         # Pa = N/m^2
rho  = 1500        # kg/m^3
Mate = ElasticMaterial1D(Emod, rho)
# Body
x_P = np.linspace(1e-5, TrussLength, NoMP)
V_P = np.ones(NoMP) * CrossSection * TrussLength/NoMP
Body = MPMBody1D(np.array([x_P, V_P]).T, Mate)
# Numerical Damping
mue = 0.0

In [3]:
# Time step estimation
cp = Grid.dx / np.sqrt(Emod/rho)
# Target elongation
dl = (9.81 * rho)/(2.0 * Emod) * TrussLength * TrussLength

In [4]:
# Initialization
t    = 0.0                                  # Global Time
# Grid
Grid.m  = np.zeros(Grid.NoNodes)            # Vector of Nodal Masses
Grid.mv = np.zeros(Grid.NoNodes)            # Vector of Nodal Momentum
Grid.v  = np.zeros(Grid.NoNodes)            # Vector of Nodal Velocities
Grid.a  = np.zeros(Grid.NoNodes)            # Vector of Nodal Accelearations
Grid.f  = np.zeros(Grid.NoNodes)            # Vector of Nodal Forces
# Body
Body.m  = Body.V * Body.Mate.rho            # Vector of Particle Masses
Body.v  = np.zeros(Body.NoMP)               # Vector of Particle Velocities
Body.mv = Body.m * Body.v                   # Vector of Partilce Momentum
Body.a  = np.zeros(Body.NoMP)               # Vector of Partilce Accelerations
Body.F  = np.ones(Body.NoMP)                # Vector of Partilce Deformation Gradients (stress free reference config)
Body.S  = np.zeros(Body.NoMP)               # Vector of Particle Stresses (stress free reference config)
Body.b  = np.ones(Body.NoMP) * 9.81         # Vector of Particle Accelerations due to gravity m/s^2

In [5]:
def ExplicitTimeStep(dt, mue):
    # Reset Grid
    Grid.m   = np.zeros(Grid.NoNodes)            # Vector of Nodal Masses
    Grid.mv  = np.zeros(Grid.NoNodes)            # Vector of Nodal Momentum
    Grid.v   = np.zeros(Grid.NoNodes)            # Vector of Nodal Velocities
    Grid.a   = np.zeros(Grid.NoNodes)            # Vector of Nodal Accelearations
    Grid.f   = np.zeros(Grid.NoNodes)            # Vector of Nodal Forces
    # Shift Particle Data
    Body.x_n = Body.X
    Body.v_n = Body.v
    Body.a_n = Body.a
    Body.F_n = Body.F
    Body.V_n = Body.V
    Body.S_n = Body.S

    # Assembly
    for P in range(NoMP):
        # connectivity
        N_I = Grid.GridNodes(Body.x_n[P])
        # assembly
        Body.ExplixitRHS(P, N_I, Grid)

    # Solve (Explicit)
    Grid.a = (Grid.f - mue * Grid.mv)/Grid.m
    Grid.a[~np.isfinite(Grid.a)] = 0          # set non finite numbers (e.g. inf or nan) to zero
    Grid.v = Grid.mv/Grid.m
    Grid.v[~np.isfinite(Grid.v)] = 0          # set non finite numbers (e.g. inf or nan) to zero
    Grid.v = Grid.v + Grid.a * dt
    # Boundary Conditions (fixed left hand side)
    Grid.a[0] = 0
    Grid.v[0] = 0

    # Update
    for P in range(NoMP):
        # connectivity
        N_I = Grid.GridNodes(Body.x_n[P])
        # update
        Body.USL(P, N_I, Grid, dt)

In [6]:
Time, NoSteps = 0, 10000
x_t = np.zeros((NoSteps,4))
for t in range(NoSteps):
    Time += dt
    ExplicitTimeStep(dt, mue)
    x_t[t] = np.array([Time, Body.X[-1], Body.StrainEnergy(), Body.KineticEnergy()])

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x_t[:,0], y=x_t[:,1],
    line=dict(color='royalblue', width=2))
    )
fig.update_layout(
    title='Truss-Lenght over Time',
    xaxis_title='time in s',
    yaxis_title='length in m'
    )
fig.show()

In [7]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x_t[:,0], y=x_t[:,2],
    mode='lines',
    name='Strain Energy'
    ))
fig.add_trace(go.Scatter(
    x=x_t[:,0], y=x_t[:,3],
    mode='lines',
    name='Kinetic Energy'
    ))
fig.add_trace(go.Scatter(
    x=x_t[:,0], y=x_t[:,2]+x_t[:,3],
    mode='lines',
    name='Total Energy'
    ))
fig.update_layout(
    title='Energy over Time',
    xaxis_title='time in s',
    yaxis_title='energy'
    )
fig.show()

In [7]:
print(dl)
print(x_t[-1,1]-TrussLength)

0.00029430000000000005
0.0002972376200958693


In [8]:
# NoMP, NoCells, u
data = np.array([
    [50, 1, 0.0029419606870446924],
    [50, 3, 0.0009809975466645027],
    [50, 6, 0.0006679994576182235],
    [50, 11, 0.0006068314007743048],
    [50, 21, 0.0006026618362144076],
    [50, 51, 0.0006000933024597754],
    [50, 81, 0.0006215417166090531],
    [50, 111, 0.000626491093039]
])
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=data[:,1], y=data[:,2],
    line=dict(color='royalblue', width=2))
    )
fig.update_layout(
    title='Convergency for Numbers of Cells',
    xaxis_title='Number of Cells',
    yaxis_title='Final Elongation'
    )
fig.show()

In [12]:
# NoMP, NoCells, u
data = np.array([
    [5, 11, 0.0007528157123913581],
    [10, 11, 0.0005703161898724107],
    [20, 11, 0.0006173523716542229],
    [50, 11, 0.0006068314007743048],
    [100, 11, 0.0006173464774665571]
    ])
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=data[:,0], y=data[:,2],
    line=dict(color='royalblue', width=2))
    )
fig.update_layout(
    title='Convergency for Numbers of MaterialPoints',
    xaxis_title='Number of MP',
    yaxis_title='Final Elongation'
    )
fig.show()